The code used here is to identify the diseased tomato fruit as a part of our project. The reference code is taken from google colab dog vs cat example.  Please suggest some applications on health care side the same code can be replicated to that healthcare dataset. Mam please update dat

In [ ]:
from zipfile import ZipFile
file_name="DATA1.zip"
with ZipFile(file_name,'r')as zip:
    zip. extractall ()
    print('Done')

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_dir = os.path.join(os.path.dirname(file_name), 'DATA1')
train_dir = os.path.join(base_dir, 'TRAIN')
validation_dir = os.path.join(base_dir, 'VALIDATION')

train_diseased_dir = os.path.join(train_dir, 'DISEASED')  # directory with our training diseased pictures
train_normal_dir = os.path.join(train_dir, 'NORMAL')  # directory with our training normal pictures
validation_diseased_dir = os.path.join(validation_dir, 'DISEASED')  # directory with our validation diseased  pictures
validation_normal_dir = os.path.join(validation_dir, 'NORMAL')  # directory with our validation normal pictures

In [ ]:
num_diseased_tr = len(os.listdir(train_diseased_dir))
num_normal_tr = len(os.listdir(train_normal_dir))

num_diseased_val = len(os.listdir(validation_diseased_dir))
num_normal_val = len(os.listdir(validation_normal_dir))
total_train = num_diseased_tr + num_normal_tr
total_val = num_diseased_val + num_normal_val

In [ ]:
print('total training diseased images:', num_diseased_tr)
print('total training normal images:', num_normal_tr)

print('total validation diseased images:', num_diseased_val)
print('total validation normal images:', num_normal_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [ ]:
BATCH_SIZE = 10  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels

In [ ]:
train_image_generator      = ImageDataGenerator(rescale=1./255)  # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=False,
                                                              target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                              class_mode='binary')

In [ ]:
sample_training_images = next(train_data_gen)

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in ZipFile(images_arr, axes):
        ax.imshow(img)
        plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:])  # Plot images 0-4

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCHS = 100
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()